[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/suud/airtable-to-map/blob/master/airtable-to-map.ipynb)

In [1]:
AIRTABLE_URL = input("Airtable URL: ")

Airtable URL: https://airtable.com/shrKKH0NycHNWQtmp


In [2]:
!pip install geopy selenium
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 958 kB 5.0 MB/s 
     |████████████████████████████████| 138 kB 48.4 MB/s 
     |████████████████████████████████| 356 kB 47.2 MB/s 
     |████████████████████████████████| 3.6 MB 34.3 MB/s 
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 54 kB 1.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/

In [3]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="test application")

import time
def get_location_dict(geolocator, raw_location_list):
    """Create dictionary containing location info.

    {"<location_name_raw>": {"count": <int>,
                             "geo": {"lat": <str>, "lon": <str>, ...}}}
    """
    location_dict = {}
    for location_raw in raw_location_list:
        if location_raw in location_dict.keys():
            # location already exists, increase count
            location_dict[location_raw]["count"] += 1
            continue

        geo = geolocator.geocode(location_raw)
        if geo:
            location_dict[location_raw] = {"count": 1,
                                           "geo": geo.raw}
        # max 1 request per second
        # https://operations.osmfoundation.org/policies/nominatim/
        time.sleep(1)
    return location_dict

import folium
from folium.plugins import MarkerCluster
def get_folium_map(location_dict):
    """Create folium map with clustered location markers."""
    map = folium.Map(zoom_start=2)
    marker_cluster = MarkerCluster().add_to(map)
    
    for name, l in locations.items():
        coordinates = (float(l["geo"]["lat"]), float(l["geo"]["lon"]))
        for _ in range(l["count"]):
            marker = folium.Marker(location=coordinates, popup=name, tooltip=name)
            marker.add_to(marker_cluster)
    return map

from selenium import webdriver
def get_chrome_webdriver():
    """Return instance of Chrome Selenium Webdriver."""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    return webdriver.Chrome('chromedriver', options=options)

import time
from bs4 import BeautifulSoup
def get_last_column_values(airtable_url):
    """Return list of values from last column of Airtable."""
    driver = get_chrome_webdriver()
    driver.get(airtable_url)
    time.sleep(5)
    html_doc = driver.page_source
    soup = BeautifulSoup(html_doc, 'html.parser')
    last_column_cells = soup.select("div.cell.lastColumn")
    return [c.text.strip() for c in last_column_cells]

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
location_list_raw = get_last_column_values(AIRTABLE_URL)
locations = get_location_dict(geolocator, location_list_raw)
map = get_folium_map(locations)

map.save("folium_map.html")

In [5]:
map